In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
labels_df = pd.read_csv('product_attributes_updated.csv', encoding='utf-8', 
                    dialect='excel', lineterminator='\n')

Using TensorFlow backend.


In [2]:
labels_df = labels_df.drop(labels_df[labels_df.Style1=='Southwestern'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style3=='Southwestern'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style1=='French Country'].index)
labels_df = labels_df.drop(labels_df[labels_df.Style2=='French Country'].index)

In [43]:
 def image2np_array( filename ) :
    img = Image.open( filename )
    img.load()
    data = np.asarray( img, dtype='uint8' )
    return data

Random_Product=labels_df['ProductID'].sample().iloc[0]

try:
    img = image2np_array('..\\ml_150x150\\P994212.jpg').reshape(1,150,150,3)
except (IOError, ValueError) as e:
    print('{0}'.format(e))
print ('true labels')
labels_df[labels_df['ProductID']==Random_Product]

true labels


,ProductID,EID,PID,VID,Department,Category,Type,Product Title,Finish1,Treatment,Hanging Method,Shade Color,Shade Shape,Style1,Style2,Style3,Tier
18380,P363571,215288.0,P363571,4036-AN,Ceiling Lights,Chandeliers,Chandeliers,Chandelier with White Shades in Antique Nickel...,Satin Nickel / Satin Chrome,Antique,Chain Hung,White Tones,Empire,Traditional,Classical,NaN,1.0


In [44]:
style_keywords=['Bronze','Brown','Satin','Chrome','Black','Brass','Gold','Polished','White','Silver','Copper','Nickel','Sconces','Bathroom','Outdoor','Pendant','Mini-Pendant','Chandeliers','Table','Flushmount','Semi-Flushmount','Multi-Light','Post','Swing','Mini-Chandeliers','Hanging','Island','Floor','Crystal','Wall','Sconce','Lamp']
style_dummy=[0]*(len(style_keywords))
for x in style_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        style_dummy[style_keywords.index(x)]=1
style_dummy

In [50]:
treatment_keywords=['Bronze','Brown','Satin','Polished','Brushed','Antique','Oil','Matte','Patina','Textured','Wall','Sconce','Lamp','Nickel']
treatment_dummy=[0]*(len(treatment_keywords))
for x in treatment_keywords:
    if x in labels_df[labels_df['ProductID']==Random_Product]['Product Title'].values[0].split(): 
        treatment_dummy[treatment_keywords.index(x)]=1
treatment_dummy

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [51]:
style_model = load_model('style_classifier.h5')
style_prediction=style_model.predict([img,np.array([style_dummy])], batch_size=1)

treatment_model = load_model('Treatment_classifier.h5')
treatment_prediction=treatment_model.predict([img,np.array([treatment_dummy])], batch_size=1)

In [ ]:
style_model = load_model('style_classifier.h5')
style_prediction=model.predict([img,style_keywords], batch_size=1)

treatment_model = load_model('Treatment_classifier.h5')


type_model = load_model('Type_classifier.h5')


finish_model = load_model('Finish1_classifier.h5')


shade_color_model = load_model('Shade_Color_classifier.h5')


shade_shape_model = load_model('Shade_Shape_classifier.h5')


hanging_metod_model = load_model('Hanging_metod_classifier.h5')


tier_model = load_model('Tier_classifier.h5')


In [52]:
style_columns=pd.get_dummies(labels_df['Style1']).columns
style_prediction_df = pd.DataFrame(data=style_prediction, columns=style_columns)
style_prediction_df.loc[:, (style_prediction_df > 0.3).any(axis=0)]

,Contemporary / Modern
0,0.335568


In [54]:
treatment_columns=pd.get_dummies(labels_df['Treatment']).columns
treatment_prediction_df = pd.DataFrame(data=treatment_prediction, columns=treatment_columns)
treatment_prediction_df.loc[:, (treatment_prediction_df > 0.3).any(axis=0)]

ValueError: Shape of passed values is (8, 1), indices imply (21, 1)